In [ ]:
# This notebook exemplifies the generation of ROC curves to test indicator detection performance

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# Read performance evaluation results and plot f-score/accuracy vs threshold
df_results=pd.read_excel('bc_evaluation_results.xlsx') 

#Calculate true positive rate and false positive rates
TP = df_results['True Positive']
FP = df_results['False Positive']
TN = df_results['True Negative']
FN = df_results['False Negative']

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
df_results['True Positive Rate'] = TPR
df_results['False Positive Rate'] = FPR
df_results['True Negative Rate'] = TNR
df_results['Sensitivity'] = TPR
df_results['Specificity'] = TNR
df_results['TPR-FPR'] = TPR-FPR

In [ ]:
df_results[df_results['Indicator']=='SPI12']

In [ ]:
df_results[df_results['Indicator']=='SPEI12']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np
from __future__ import division 
import numpy as np
import matplotlib.pyplot as plt
optimal_thresholds=thresholds_df[['Indicator','Hazard','Type']]
optimal_thresholds['Threshold']=np.nan
def accuracy_plot(df_results, thresholds_df):
    df_results_plot = df_results.copy()
    df_results_plot = df_results_plot.dropna()
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('12-Month Precipitation', '12-Mon. Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Daily Precipitation', 'Daily Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Max. Daily Temperature', 'Max. Daily Temp.')
    
    top_indicators = ['HWI', 'SSI12', 'SPEI12', 'SSI12', 'SPEI12']
    bottom_indicators = ['Max. Daily Temp.', 'Daily Discharge', '12-Mon. Prec.', 'Daily Prec.','DFI']
    hazards = ['Heatwave', 'Drought', 'Drought', 'Flood', 'Flood','Heatwave', 'Flood', 'Drought', 'Flood','Flood']
    labels = ['Value', 'Value', 'Value', 'Value','Value', '°C', 'm3/s', 'mm', 'mm', 'Value']
    xmin = [-25, -3, -3, -4, 0, 0, 0, 0]
    xmax = [10, 3, 3, 3, 40, 3000, 1300, 200]
    
    fig, axs = plt.subplots(2, 5, figsize=(14, 6))
    axs = axs.flatten()
    sns.set(style="whitegrid")

    for i, indicator in enumerate(top_indicators+bottom_indicators):
        indicator_data = df_results_plot[df_results_plot['Indicator'] == indicator]
        hazard = hazards[i]
        ax = axs[i]
        hazard_data = indicator_data[indicator_data['Hazard'] == hazard]
        hazard_data = hazard_data.sort_values(by='Threshold')
        indicator_df = thresholds_df[thresholds_df['Indicator'] == indicator]
        hazard_df = indicator_df[indicator_df['Hazard'] == hazard]
        #ttype = hazard_df.Type.iloc[0]
        # Find the threshold corresponding to maximum accuracy
        #target_accuracy = 100
        #op_thre = hazard_data.loc[(np.abs(hazard_data['Accuracy'] - target_accuracy)).idxmin(), 'Threshold']
        max_idx = np.argmax(hazard_data['Accuracy'])
        op_thre = hazard_data['Threshold'].iloc[max_idx]
        optimal_thresholds['Threshold'].iloc[i]= op_thre
        optimal_thresholds['Hazard'].iloc[i]= hazard
        optimal_thresholds['Indicator'].iloc[i]= indicator
        if hazard=='Drought':
            optimal_thresholds['Type'].iloc[i]= 'Upper'
        else:
            optimal_thresholds['Type'].iloc[i]= 'Lower'
        #op_thre = hazard_df.Threshold.iloc[0]
        #op_thre = interpolated_intercept(hazard_data['Threshold'].values, hazard_data['Accuracy'].values, hazard_data['Recall'].values)
        ax.axvline(x=op_thre, color='black', linestyle='dashed')
        ax.plot(hazard_data['Threshold'], hazard_data['Accuracy']*100, color='blue')
        #ax.plot(hazard_data['Threshold'], hazard_data['Recall']*100, color='red', linestyle='-')


        ax.set_title(indicator+' ('+hazard+')')
        ax.set_xlabel('Threshold'+' ('+labels[i]+')')
        ax.set_xlim([hazard_data['Threshold'].min(), hazard_data['Threshold'].max()])
        ax.set_ylabel('Accuracy (%)')
        ax.set_ylim([-5, 105])
    
    plt.tight_layout()
    plt.show()
    return optimal_thresholds



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np
from __future__ import division 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import auc
optimal_indicators=thresholds_df[['Indicator','Hazard','Type']]
optimal_indicators['AUC']=np.nan
optimal_indicators['Threshold']=np.nan
df_results_plot = df_results.copy()
#df_results_plot = df_results_plot.dropna()

#df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('12-Month Precipitation', '12-Mon. Prec.')
#df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Daily Precipitation', 'Daily Prec.')
#df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Max. Daily Temperature', 'Max. Daily Temp.')

indicators=thresholds_df['Indicator'].values
hazards=thresholds_df['Hazard'].values
for i, indicator in enumerate(indicators):
    indicator_data = df_results_plot[df_results_plot['Indicator'] == indicator]
    hazard = hazards[i]
    #print(indicator, hazard)
    hazard_data = indicator_data[indicator_data['Hazard'] == hazard]
    hazard_data = hazard_data.sort_values(by='Threshold')
    #indicator_df = thresholds_df[thresholds_df['Indicator'] == indicator]
    #hazard_df = indicator_df[indicator_df['Hazard'] == hazard]
    #ttype = hazard_df.Type.iloc[0]
    # Find the threshold 
    #op_thre = hazard_data.loc[(np.abs(hazard_data['Sensitivity'] - hazard_data['Specificity'])).idxmin(), 'Threshold']
    auc1=auc(1-hazard_data['Specificity'], hazard_data['Sensitivity'])
    #optimal_indicators['Threshold'].iloc[i]= op_thre
    optimal_indicators['AUC'].iloc[i]= auc1
    optimal_indicators['Hazard'].iloc[i]= hazard
    optimal_indicators['Indicator'].iloc[i]= indicator
    if hazard=='Drought':
        optimal_indicators['Type'].iloc[i]= 'Upper'
    else:
        optimal_indicators['Type'].iloc[i]= 'Lower'
    auc1=auc(1-hazard_data['Specificity'], hazard_data['Sensitivity'])
    tpr = hazard_data['True Positive Rate'].values
    fpr = hazard_data['False Positive Rate'].values
    youden_index = tpr + (1 - fpr) - 1
    optimal_idx = np.argmax(youden_index)
    optimal_threshold = hazard_data['Threshold'].values[optimal_idx]
    optimal_indicators['Threshold'].iloc[i]= optimal_threshold
    print(auc1)



In [ ]:
optimal_indicators.sort_values(by='AUC', ascending=False)

In [ ]:
indicator

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def rpcurve_plot(df_results, thresholds_df):
    df_results_plot = df_results.copy()
    df_results_plot = df_results_plot.dropna()
    #df_results_plot = df_results_plot.sort_values(by='Hazard')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('12-Month Precipitation', '12-Mon. Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Daily Precipitation', 'Daily Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Max. Daily Temperature', 'Max. Daily Temp.')
    # Define the order of indicators at the top and bottom
    top_indicators = ['HWI', 'SSI12', 'SPEI12', 'SSI12', 'SPEI12']
    bottom_indicators = ['Max. Daily Temp.', 'Daily Discharge', '12-Mon. Prec.', 'Daily Prec.','DFI']
    hazards = ['Heatwave', 'Drought', 'Drought', 'Flood', 'Flood','Heatwave', 'Flood', 'Drought', 'Flood','Flood']
    labels = ['Value', 'Value', 'Value', 'Value','Value', '°C', 'm3/s', 'mm', 'mm', 'Value']
    xmin = [-25, -3, -3, -4, 0, 0, 0, 0]
    xmax = [10, 3, 3, 3, 40, 3000, 1300, 200]

    # Create subplots with 2 rows and 2 columns
    fig, axs = plt.subplots(2, 5, figsize=(14, 6))
    axs = axs.flatten()

    # Set the plot style to whitegrid
    sns.set(style="whitegrid")

    # Iterate over the indicators and plot boxplots
    for i, indicator in enumerate(top_indicators+bottom_indicators):
        # Filter the data for the current indicator
        indicator_data = df_results_plot[df_results_plot['Indicator'] == indicator]
        hazard = hazards[i]
        ax = axs[i]
        hazard_data = indicator_data[indicator_data['Hazard'] == hazard]
        hazard_data = hazard_data.sort_values(by='Threshold')
        indicator_df = thresholds_df[thresholds_df['Indicator'] == indicator]
        hazard_df = indicator_df[indicator_df['Hazard'] == hazard]
        #ttype = hazard_df.Type.iloc[0]
        #print(hazard_df)
        #op_thre = hazard_df.Threshold.iloc[0]
        ax.plot(hazard_data['Recall'], hazard_data['Precision'], color='blue')
        
        # Set the title for the subplot
        ax.set_title(indicator+' ('+hazard+')')

        # Set the x-axis label
        ax.set_xlabel('Recall')
        ax.set_xlim([0, 1.05])
        #ax.set_xlim([xmin[i], xmax[i]])

        # Set the y-axis labels and limits
        ax.set_ylabel('Precision')
        ax.set_ylim([0, 1.05])

        # Add legend for the threshold lines
        #ax.legend(loc='best')

        #ax.axvline(x=op_thre, color='black', linestyle='dashed')


    # Adjust the spacing between subplots
    plt.tight_layout()

    # Show the plot
    plt.show()


In [ ]:
rpcurve_plot(df_results, thresholds_df)

In [ ]:
list(df_results)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def rp_plot(df_results, thresholds_df):
    df_results_plot = df_results.copy()
    df_results_plot = df_results_plot.dropna()
    #df_results_plot = df_results_plot.sort_values(by='Hazard')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('12-Month Precipitation', '12-Mon. Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Daily Precipitation', 'Daily Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Max. Daily Temperature', 'Max. Daily Temp.')
    # Define the order of indicators at the top and bottom
    top_indicators = ['HWI', 'SSI12', 'SPEI12', 'SSI12', 'SPEI12']
    bottom_indicators = ['Max. Daily Temp.', 'Daily Discharge', '12-Mon. Prec.', 'Daily Prec.','DFI']
    hazards = ['Heatwave', 'Drought', 'Drought', 'Flood', 'Flood','Heatwave', 'Flood', 'Drought', 'Flood','Flood']
    labels = ['Value', 'Value', 'Value', 'Value','Value', '°C', 'm3/s', 'mm', 'mm', 'Value']
    xmin = [-25, -3, -3, -4, 0, 0, 0, 0]
    xmax = [10, 3, 3, 3, 40, 3000, 1300, 200]

    # Create subplots with 2 rows and 2 columns
    fig, axs = plt.subplots(2, 5, figsize=(14, 6))
    axs = axs.flatten()

    # Set the plot style to whitegrid
    sns.set(style="whitegrid")

    # Iterate over the indicators and plot boxplots
    for i, indicator in enumerate(top_indicators+bottom_indicators):
        # Filter the data for the current indicator
        indicator_data = df_results_plot[df_results_plot['Indicator'] == indicator]
        hazard = hazards[i]
        ax = axs[i]
        hazard_data = indicator_data[indicator_data['Hazard'] == hazard]
        hazard_data = hazard_data.sort_values(by='Threshold')
        indicator_df = thresholds_df[thresholds_df['Indicator'] == indicator]
        hazard_df = indicator_df[indicator_df['Hazard'] == hazard]
        #ttype = hazard_df.Type.iloc[0]
        #op_thre = hazard_df.Threshold.iloc[0]
        ax.plot(hazard_data['Threshold'], hazard_data['Precision']*100, color='blue')
        ax.plot(hazard_data['Threshold'], hazard_data['Recall']*100, color='red')
        # Set the title for the subplot
        ax.set_title(indicator+' ('+hazard+')')

        # Set the x-axis label
        ax.set_xlabel(labels[i])
        #ax.set_xlim([xmin[i], xmax[i]])

        # Set the y-axis labels and limits
        ax.set_ylabel('Precision (%)')
        ax.set_ylim([0, 105])

        # Add legend for the threshold lines
        #ax.axvline(x=op_thre, color='black', linestyle='dashed')
    fig.legend(handles=[
        plt.Line2D([], [], linestyle='-', color='blue', label='Precision'),
        plt.Line2D([], [], linestyle='-', color='red', label='Recall'),
        plt.Line2D([], [], linestyle='--', color='black', label='Selected Threshold')
    ], loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.05))

    # Adjust the spacing between subplots
    plt.tight_layout()

    # Show the plot
    plt.show()


In [ ]:
rp_plot(df_results, thresholds_df)

In [ ]:
list(df_results)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def rates_plot(df_results):
    df_results_plot = df_results.copy()
    df_results_plot = df_results_plot.dropna()
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('12-Month Precipitation', '12-Mon. Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Daily Precipitation', 'Daily Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Max. Daily Temperature', 'Max. Daily Temp.')
    #top_indicators = ['HWI', 'SSI12', 'SPEI12', 'SSI12', 'SPEI12']
    #bottom_indicators = ['Max. Daily Temp.', 'Daily Discharge', '12-Mon. Prec.', 'Daily Prec.', 'DFI']
    hazards = ['Heatwave', 'Drought', 'Flood']
    indicators = ['HWI', 'SPEI12', 'DFI']
    #labels = ['Value', 'Value', 'Value', 'Value','Value', '°C', 'm3/s', 'mm', 'mm', 'Value']
    xmin = [-25, -3, -3, -4, 0, 0, 0, 0]
    xmax = [10, 3, 3, 3, 40, 3000, 1300, 200]

    # Create subplots with 1 rows and 3 columns
    fig, axs = plt.subplots(1, 3, figsize=(10, 3))
    axs = axs.flatten()

    sns.set(style="ticks")

    for i, indicator in enumerate(indicators):
        indicator_data = df_results_plot[df_results_plot['Indicator'] == indicator]
        hazard = hazards[i]
        ax = axs[i]
        hazard_data = indicator_data[indicator_data['Hazard'] == hazard]
        hazard_data = hazard_data.sort_values(by='Threshold')

        tpr = hazard_data['True Positive Rate'].values
        fpr = hazard_data['False Positive Rate'].values
        thr = hazard_data['Threshold'].values
        
        if indicator=='SPEI12':
            tpr = np.append(tpr, 1)
            tpr = np.insert(tpr, 0, 0)
            fpr = np.append(fpr, 1)
            fpr = np.insert(fpr, 0, 0)
            thr = np.append(thr, np.nan)
            thr = np.insert(thr, 0, np.nan)
        else:
            tpr = np.append(tpr, 0)
            tpr = np.insert(tpr, 0, 1)
            fpr = np.append(fpr, 0)
            fpr = np.insert(fpr, 0, 1)
            thr = np.append(thr, np.nan)
            thr = np.insert(thr, 0, np.nan)

        youden_index = tpr - fpr
        optimal_idx = np.argmax(youden_index)
        #optimal_idx = np.argmax(hazard_data['F-Score'].values)
        #optimal_idx = np.argmax(hazard_data['Precision'].values)
        #optimal_idx = np.argmax(hazard_data['Accuracy'].values)

        optimal_threshold = thr[optimal_idx]
        optimal_tpr = tpr[optimal_idx]
        optimal_fpr = fpr[optimal_idx]
        auc1 = auc(fpr, tpr)
        ax.plot(fpr*100, tpr*100, color='blue')
        ax.plot(optimal_fpr*100, optimal_tpr*100, marker='o', markersize=8, color='red')
        ax.text(60, 20, r'$\mathrm{t}_{\mathrm{opt}}$'+f': {optimal_threshold:.2f}', color='black')
        ax.text(60, 10, f'AUC: {auc1:.2f}', color='black')
        ax.set_title(indicator+' ('+hazard+')')
        ax.set_xlabel('False Positive Rate (%)')
        ax.set_xlim([-5, 105])
        ax.set_ylabel('True Positive Rate (%)')
        ax.set_ylim([-5, 105])

    plt.tight_layout()
    plt.show()


In [ ]:
rates_plot(df_results)

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import auc
units_df=pd.read_excel(r'C:\Users\MarlonVieiraPassos\OneDrive - SEI\Documents\HydroHazards\Codes\SMHI API\Performance\input\Units.xlsx')

def rates_plot2(df_results):
    df_results_plot = df_results.copy()
    df_results_plot = df_results_plot.dropna()
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('12-Month Precipitation', '12-Mon. Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('6-Month Precipitation', '6-Mon. Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('3-Month Precipitation', '3-Mon. Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('HWI', 'HWI_S')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('HWI-EDO', 'HWI_E')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Daily Precipitation', 'Daily Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Daily Discharge', 'Daily Disc.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Max. Daily Temperature', 'Max. Daily Temp.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Min. Daily Temperature', 'Min. Daily Temp.')
    #top_indicators = ['HWI', 'SSI12', 'SPEI12', 'SSI12', 'SPEI12']
    #bottom_indicators = ['Max. Daily Temp.', 'Daily Discharge', '12-Mon. Prec.', 'Daily Prec.', 'DFI']
    hazards = ['Heatwave', 'Drought', 'Flood']
    #indicators = ['HWI', 'SPEI12', 'DFI']
    labels = {'Heatwave':'°C', 'Discharge':'m3/s', 'Precipitation':'mm', 'Other':''}
    #xmin = [-25, -3, -3, -4, 0, 0, 0, 0]
    #xmax = [10, 3, 3, 3, 40, 3000, 1300, 200]
    lcolors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan', 'magenta', 'lime', 'teal']
    lstyles = ['-', '--', '-.', ':','-', '--', '-.', ':','-', '--', '-.', ':']
    auc_df=pd.DataFrame(columns=['Indicator','Hazard', 'AUC', 'Opt. Threshold','Opt. TPR','Opt. FPR'])
    # Create subplots with 3 rows and 1 column
    fig, axs = plt.subplots(3, 1, figsize=(10, 15))
    axs = axs.flatten()

    sns.set(style="ticks")
    for j, hazard in enumerate(hazards):
        indicators_df= df_results_plot[df_results_plot['Hazard'] == hazard]
        indicators_df  = indicators_df.sort_values(by='Indicator')
        if hazard=='Heatwave':
            indicators_df  = indicators_df.sort_values(by='Indicator', ascending=False)
        indicators = indicators_df.Indicator.unique()
        for i, indicator in enumerate(indicators):
            #indicator_data = df_results_plot[df_results_plot['Indicator'] == indicator]
            indicator_data = indicators_df[indicators_df['Indicator'] == indicator]
            ax = axs[j]
            hazard_data = indicator_data[indicator_data['Hazard'] == hazard]
            hazard_data = hazard_data.sort_values(by='Threshold')

            tpr = hazard_data['True Positive Rate'].values
            fpr = hazard_data['False Positive Rate'].values
            thr = hazard_data['Threshold'].values

            if hazard=='Drought':
                tpr = np.append(tpr, 1)
                tpr = np.insert(tpr, 0, 0)
                fpr = np.append(fpr, 1)
                fpr = np.insert(fpr, 0, 0)
                thr = np.append(thr, np.nan)
                thr = np.insert(thr, 0, np.nan)
            else:
                tpr = np.append(tpr, 0)
                tpr = np.insert(tpr, 0, 1)
                fpr = np.append(fpr, 0)
                fpr = np.insert(fpr, 0, 1)
                thr = np.append(thr, np.nan)
                thr = np.insert(thr, 0, np.nan)

            youden_index = tpr - fpr
            optimal_idx = np.argmax(youden_index)
            #optimal_idx = np.argmax(hazard_data['F-Score'].values)
            #optimal_idx = np.argmax(hazard_data['Precision'].values)
            #optimal_idx = np.argmax(hazard_data['Accuracy'].values)
            optimal_threshold = thr[optimal_idx]
            optimal_tpr = tpr[optimal_idx]
            optimal_fpr = fpr[optimal_idx]
            units=units_df.set_index('Indicator').to_dict()['Units']
            unit_txt=str(units[indicator])
            if unit_txt=='nan':
                unit_txt=''
            else:
                unit_txt=' '+unit_txt
            auc1 = auc(fpr, tpr)
            if indicator=='HWI_E':
                indicator='$\mathrm{HWI}_{\mathrm{E}}$'
            if indicator=='HWI_S':
                indicator='$\mathrm{HWI}_{\mathrm{S}}$'
            text= indicator+' ('+f'AUC= {auc1:.2f}, '+'$\mathrm{t}_{\mathrm{opt}}$'+f'= {optimal_threshold:.2f}'+unit_txt+')'
            ax.plot(fpr*100, tpr*100, color=lcolors[i], linestyle=lstyles[i], label=text)
            ax.plot(optimal_fpr*100, optimal_tpr*100, marker='o', markersize=4, color=lcolors[i])
            #ax.text(60, 20, r'$\mathrm{t}_{\mathrm{opt}}$'+f': {optimal_threshold:.2f}', color='black')
            #ax.text(60, 10, f'AUC: {auc1:.2f}', color='black')
            row= {'Indicator': indicator, 'Hazard': hazard, 'AUC':auc1, 
                    'Opt. Threshold':optimal_threshold, 'Opt. TPR':optimal_tpr, 'Opt. FPR': optimal_fpr}
            auc_df = auc_df.append(row, ignore_index=True)
        ax.set_title('ROC curves of '+hazard.lower()+' indicators')
        ax.set_xlabel('False Positive Rate (%)')
        ax.set_xlim([-5, 105])
        ax.set_ylabel('True Positive Rate (%)')
        ax.set_ylim([-5, 105])
        ax.legend(ncol=2, fontsize=12)

    plt.tight_layout()
    plt.show()
    return auc_df


In [ ]:
auc_df=rates_plot2(df_results)

In [ ]:
auc_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import auc
def rates_plot_hw(df_results):
    df_results_plot = df_results.copy()
    df_results_plot = df_results_plot.dropna()
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('HWI', 'HWI (SMHI)')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('HWI-EDO', 'HWI (EDO)')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('12-Month Precipitation', '12-Mon. Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Daily Precipitation', 'Daily Prec.')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Max. Daily Temperature', 'Max. Daily Temp. (°C)')
    df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Min. Daily Temperature', 'Min. Daily Temp. (°C)')
    #top_indicators = ['HWI', 'SSI12', 'SPEI12', 'SSI12', 'SPEI12']
    #bottom_indicators = ['Max. Daily Temp.', 'Daily Discharge', '12-Mon. Prec.', 'Daily Prec.', 'DFI']
    hazards = ['Heatwave', 'Heatwave', 'Heatwave', 'Heatwave']
    indicators = ['HWI (SMHI)', 'HWI (EDO)', 'Max. Daily Temp. (°C)', 'Min. Daily Temp. (°C)']
    #labels = ['Value', 'Value', 'Value', 'Value','Value', '°C', 'm3/s', 'mm', 'mm', 'Value']
    xmin = [-25, -3, -3, -4, 0, 0, 0, 0]
    xmax = [10, 3, 3, 3, 40, 3000, 1300, 200]

    # Create subplots with 1 rows and 4 columns
    fig, axs = plt.subplots(1, 4, figsize=(12, 3))
    axs = axs.flatten()

    sns.set(style="ticks")

    for i, indicator in enumerate(indicators):
        indicator_data = df_results_plot[df_results_plot['Indicator'] == indicator]
        hazard = hazards[i]
        ax = axs[i]
        hazard_data = indicator_data[indicator_data['Hazard'] == hazard]
        hazard_data = hazard_data.sort_values(by='Threshold')

        tpr = hazard_data['True Positive Rate'].values
        tpr = np.append(tpr, 0)
        tpr = np.insert(tpr, 0, 1)
        fpr = hazard_data['False Positive Rate'].values
        fpr = np.append(fpr, 0)
        fpr = np.insert(fpr, 0, 1)
        thr = hazard_data['Threshold'].values
        thr = np.append(thr, np.nan)
        thr = np.insert(thr, 0, np.nan)
        #print(indicator)
        #print(tpr)
        #print(fpr)
        youden_index = tpr + (1 - fpr) - 1
        optimal_idx = np.argmax(youden_index)
        #optimal_idx = np.argmax(hazard_data['Accuracy'].values)

        optimal_threshold = thr[optimal_idx]
        optimal_tpr = tpr[optimal_idx]
        optimal_fpr = fpr[optimal_idx]
        #auc1=auc(1-hazard_data['Specificity'], hazard_data['Sensitivity'])
        auc1 = auc(fpr, tpr)
        ax.plot(fpr*100, tpr*100, color='blue')
        ax.plot(optimal_fpr*100, optimal_tpr*100, marker='o', markersize=8, color='red')
        ax.text(60, 20, r'$\mathrm{t}_{\mathrm{opt}}$'+f': {optimal_threshold:.2f}', color='black')
        ax.text(60, 10, f'AUC: {auc1:.2f}', color='black')
        ax.set_title(indicator)
        ax.set_xlabel('False Positive Rate (%)')
        ax.set_xlim([-5, 105])
        ax.set_ylabel('True Positive Rate (%)')
        ax.set_ylim([-5, 105])

    plt.tight_layout()
    plt.show()


In [ ]:
rates_plot_hw(df_results) 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def rates_plot_dr(df_results):
    df_results_plot = df_results.copy()
    df_results_plot = df_results_plot.dropna()
    df_results_plot = df_results_plot[df_results_plot['Hazard']=='Drought']
    df_results_plot = df_results_plot.sort_values(by='Indicator')
    #indicators = df_results_plot.Indicator.unique()
    indicators = ['3-Month Precipitation', 'SPEI3', 'SPI3', 'SSI3',
    '6-Month Precipitation', 'SPEI6', 'SPI6', 'SSI6',
    '12-Month Precipitation', 'SPEI12', 'SPI12', 'SSI12',
    ]
    hazards = ['Drought' for indicator in indicators]
    #df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('12-Month Precipitation', '12-Mon. Prec.')
    #df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Daily Precipitation', 'Daily Prec.')
    #hazards = ['Heatwave', 'Heatwave', 'Heatwave', 'Heatwave']
    #indicators = ['HWI (SMHI)', 'HWI (EDO)', 'Max. Daily Temp. (°C)', 'Min. Daily Temp. (°C)']
    #labels = ['Value', 'Value', 'Value', 'Value','Value', '°C', 'm3/s', 'mm', 'mm', 'Value']

    # Create subplots with 3 rows and 4 columns
    fig, axs = plt.subplots(3, 4, figsize=(12, 9))
    axs = axs.flatten()

    sns.set(style="ticks")

    for i, indicator in enumerate(indicators):
        indicator_data = df_results_plot[df_results_plot['Indicator'] == indicator]
        hazard = hazards[i]
        ax = axs[i]
        hazard_data = indicator_data[indicator_data['Hazard'] == hazard]
        hazard_data = hazard_data.sort_values(by='Threshold')

        tpr = hazard_data['True Positive Rate'].values
        tpr = np.append(tpr, 1)
        tpr = np.insert(tpr, 0, 0)
        fpr = hazard_data['False Positive Rate'].values
        fpr = np.append(fpr, 1)
        fpr = np.insert(fpr, 0, 0)
        thr = hazard_data['Threshold'].values
        thr = np.append(thr, np.nan)
        thr = np.insert(thr, 0, np.nan)

        youden_index = tpr + (1 - fpr) - 1
        optimal_idx = np.argmax(youden_index)
        #optimal_idx = np.argmax(hazard_data['Accuracy'].values)

        optimal_threshold = thr[optimal_idx]
        optimal_tpr = tpr[optimal_idx]
        optimal_fpr = fpr[optimal_idx]
        auc1=auc(1-hazard_data['Specificity'], hazard_data['Sensitivity'])
        ax.plot(fpr*100, tpr*100, color='blue')
        ax.plot(optimal_fpr*100, optimal_tpr*100, marker='o', markersize=8, color='red')
        ax.text(60, 20, r'$\mathrm{t}_{\mathrm{opt}}$'+f': {optimal_threshold:.2f}', color='black')
        ax.text(60, 10, f'AUC: {auc1:.2f}', color='black')
        ax.set_title(indicator)
        ax.set_xlabel('False Positive Rate (%)')
        ax.set_xlim([-5, 105])
        ax.set_ylabel('True Positive Rate (%)')
        ax.set_ylim([-5, 105])

    plt.tight_layout()
    plt.show()


In [ ]:
rates_plot_dr(df_results)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import auc
def rates_plot_fl(df_results):
    df_results_plot = df_results.copy()
    df_results_plot = df_results_plot.dropna()
    df_results_plot = df_results_plot[df_results_plot['Hazard']=='Flood']
    df_results_plot = df_results_plot.sort_values(by='Indicator')
    #indicators = df_results_plot.Indicator.unique()
    indicators = ['Daily Precipitation', 'SPEI3', 'SPI3', 'SSI3',
    'Daily Discharge', 'SPEI6', 'SPI6', 'SSI6',
    'DFI', 'SPEI12', 'SPI12', 'SSI12',
    ]
    hazards = ['Flood' for indicator in indicators]
    #df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('12-Month Precipitation', '12-Mon. Prec.')
    #df_results_plot['Indicator'] = df_results_plot['Indicator'].replace('Daily Precipitation', 'Daily Prec.')
    #hazards = ['Heatwave', 'Heatwave', 'Heatwave', 'Heatwave']
    #indicators = ['HWI (SMHI)', 'HWI (EDO)', 'Max. Daily Temp. (°C)', 'Min. Daily Temp. (°C)']
    #labels = ['Value', 'Value', 'Value', 'Value','Value', '°C', 'm3/s', 'mm', 'mm', 'Value']

    # Create subplots with 3 rows and 4 columns
    fig, axs = plt.subplots(3, 4, figsize=(12, 9))
    axs = axs.flatten()

    sns.set(style="ticks")

    for i, indicator in enumerate(indicators):
        indicator_data = df_results_plot[df_results_plot['Indicator'] == indicator]
        hazard = hazards[i]
        ax = axs[i]
        hazard_data = indicator_data[indicator_data['Hazard'] == hazard]
        hazard_data = hazard_data.sort_values(by='Threshold')

        tpr = hazard_data['True Positive Rate'].values
        tpr = np.append(tpr, 0)
        tpr = np.insert(tpr, 0, 1)
        fpr = hazard_data['False Positive Rate'].values
        fpr = np.append(fpr, 0)
        fpr = np.insert(fpr, 0, 1)
        thr = hazard_data['Threshold'].values
        thr = np.append(thr, np.nan)
        thr = np.insert(thr, 0, np.nan)

        youden_index = tpr + (1 - fpr) - 1
        optimal_idx = np.argmax(youden_index)
        #optimal_idx = np.argmax(hazard_data['Accuracy'].values)

        optimal_threshold = thr[optimal_idx]
        optimal_tpr = tpr[optimal_idx]
        optimal_fpr = fpr[optimal_idx]
        auc1=auc(1-hazard_data['Specificity'], hazard_data['Sensitivity'])
        ax.plot(fpr*100, tpr*100, color='blue')
        ax.plot(optimal_fpr*100, optimal_tpr*100, marker='o', markersize=8, color='red')
        ax.text(60, 20, r'$\mathrm{t}_{\mathrm{opt}}$'+f': {optimal_threshold:.2f}', color='black')
        ax.text(60, 10, f'AUC: {auc1:.2f}', color='black')
        ax.set_title(indicator)
        ax.set_xlabel('False Positive Rate (%)')
        ax.set_xlim([-5, 105])
        ax.set_ylabel('True Positive Rate (%)')
        ax.set_ylim([-5, 105])

    plt.tight_layout()
    plt.show()


In [ ]:
rates_plot_fl(df_results) 